In [1]:
# Import libraries
import pandas as pd
import nflreadpy as nfl
import nfl_data_py as nfl_data
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
# Load NFL data for 2025 season
season = 2025
schedule = nfl.load_schedules(seasons=[season])
schedule = schedule.to_pandas()

# Select only regular season games
team_stats = nfl.load_team_stats([season, summary_level='reg'])
team_stats = team_stats.to_pandas()

In [3]:
# Add column for EPA per play
team_stats["pass_plays"] = team_stats["attempts"] + team_stats["sacks_suffered"]
team_stats["rush_plays"] = team_stats["carries"]
team_stats["total_plays"] = team_stats["pass_plays"] + team_stats["rush_plays"]
team_stats["off_epa_per_play"] = (
    (team_stats["passing_epa"] * team_stats["pass_plays"]) +
    (team_stats["rushing_epa"] * team_stats["rush_plays"])
) / team_stats["total_plays"]

In [4]:
# Add columns points for and against to team_stats
home = schedule[["season", "week", "home_team", "away_team", "home_score", "away_score"]].copy()
home["team"] = home["home_team"]
home["points_for"] = home["home_score"]
home["points_against"] = home["away_score"]

away = schedule[["season", "week", "home_team", "away_team", "home_score", "away_score"]].copy()
away["team"] = away["away_team"]
away["points_for"] = away["away_score"]
away["points_against"] = away["home_score"]

schedule_long = pd.concat([home, away], ignore_index=True)

# Merge into team_stats
team_stats = team_stats.merge(
    schedule_long[["season", "week", "team", "points_for", "points_against"]],
    on=["season", "week", "team"],
    how="left"
)

In [5]:
# Add column for turnover differential
# Offensive giveaways
team_stats["giveaways"] = (
    team_stats["passing_interceptions"] + 
    team_stats["rushing_fumbles_lost"] + 
    team_stats["receiving_fumbles_lost"] + 
    team_stats["sack_fumbles_lost"]
)

# Defensive takeaways
team_stats["takeaways"] = (
    team_stats["def_interceptions"] + 
    team_stats["fumble_recovery_opp"]   # opp fumbles recovered by your team
)

# Turnover differential
team_stats["turnover_diff"] = team_stats["takeaways"] - team_stats["giveaways"]

In [71]:
# Web scrape defensive EPA/play from https://sumersports.com/teams/defensive/
import requests
from bs4 import BeautifulSoup

url = "https://sumersports.com/teams/defensive/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Locate table
table = soup.find('table')

headers = [th.get_text(strip=True) for th in table.find_all('th')]

# Extract rows
rows = []
for tr in table.find_all('tr')[1:]:  # Skip header row
    cols = [td.get_text(strip=True) for td in tr.find_all('td')]
    if cols:  # Check row is not empty
        rows.append(cols)

df = pd.DataFrame(rows, columns=headers)
# Clean up team names (remove extra spaces)
df['team'] = df['Team'].str.replace(r'^\d+\.\s*', '', regex=True)
df['EPA/Play'] = pd.to_numeric(df['EPA/Play'], errors='coerce')

def_epa_df = df[['team', 'EPA/Play']].copy()


In [72]:
# Web scrape 3rd down conversion % (offense and defense) from https://www.teamrankings.com/
urls = {
    'third_down': 'https://www.teamrankings.com/nfl/stat/third-down-conversion-pct',
    'opponent_third_down': 'https://www.teamrankings.com/nfl/stat/opponent-third-down-conversion-pct'
}

# Function to scrape data from a given URL
def scrape_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate table
    table = soup.find('table')

    # Extract headers
    headers = [th.get_text(strip=True) for th in table.find_all('th')]

    # Extract rows
    rows = []
    for tr in table.find_all('tr')[1:]:  # Skip header
        cols = [td.get_text(strip=True) for td in tr.find_all('td')]
        if cols:  # Check row is not empty
            rows.append(cols)

    # Create a df
    df = pd.DataFrame(rows, columns=headers)

    # Clean up team names (remove extra spaces)
    df['Team'] = df['Team'].str.strip()

    return df[['Team', '2025']]

# Scrape data
third_down_df = scrape_data(urls['third_down'])
opponent_third_down_df = scrape_data(urls['opponent_third_down'])

# Merge on the 'Team' column
merged_third_down_df = pd.merge(third_down_df, opponent_third_down_df, on='Team', suffixes=('_Offense', '_Defense'))


In [73]:
# Add indicator column for winner
team_stats['winner'] = (team_stats['points_for'] > team_stats['points_against']).astype(int)

In [74]:
# Create dataframe with cumulative averages of features
def cumulative_avg(df, up_to_week):
    stats_cols = ["off_epa_per_play", "points_for", "points_against", "turnover_diff"]

    # filter games up to target week
    df_filtered = df[df["week"] <= up_to_week].copy()

    # calculate cumulative average per team
    cumu = (
        df_filtered.groupby("team")[stats_cols]
        .mean()
        .reset_index()
    )

    # rename columns
    cumu = cumu.rename(columns={col: f"{col}" for col in stats_cols})

    return cumu

In [75]:
# Create cumulative avg dataframe for weeks 1-3
cumulative_df = cumulative_avg(team_stats, up_to_week=3)

In [76]:
# Convert all team names to abbreviations
TEAM_ABBR = {
    "ARI": ["Arizona Cardinals", "Arizona"],
    "ATL": ["Atlanta Falcons", "Atlanta"],
    "BAL": ["Baltimore Ravens", "Baltimore"],
    "BUF": ["Buffalo Bills", "Buffalo"],
    "CAR": ["Carolina Panthers", "Carolina"],
    "CHI": ["Chicago Bears", "Chicago"],
    "CIN": ["Cincinnati Bengals", "Cincinnati"],
    "CLE": ["Cleveland Browns", "Cleveland"],
    "DAL": ["Dallas Cowboys", "Dallas"],
    "DEN": ["Denver Broncos", "Denver"],
    "DET": ["Detroit Lions", "Detroit"],
    "GB":  ["Green Bay Packers", "Green Bay"],
    "HOU": ["Houston Texans", "Houston"],
    "IND": ["Indianapolis Colts", "Indianapolis"],
    "JAX": ["Jacksonville Jaguars", "Jacksonville"],
    "KC":  ["Kansas City Chiefs", "Kansas City"],
    "LV":  ["Las Vegas Raiders", "Las Vegas"],
    "LAC": ["Los Angeles Chargers", "LA Chargers"],
    "LAR": ["Los Angeles Rams", "LA Rams"],
    "MIA": ["Miami Dolphins", "Miami"],
    "MIN": ["Minnesota Vikings", "Minnesota"],
    "NE":  ["New England Patriots", "New England"],
    "NO":  ["New Orleans Saints", "New Orleans"],
    "NYG": ["New York Giants", "NY Giants"],
    "NYJ": ["New York Jets", "NY Jets"],
    "PHI": ["Philadelphia Eagles", "Philadelphia"],
    "PIT": ["Pittsburgh Steelers", "Pittsburgh"],
    "SF":  ["San Francisco 49ers", "San Francisco"],
    "SEA": ["Seattle Seahawks", "Seattle"],
    "TB":  ["Tampa Bay Buccaneers", "Tampa Bay"],
    "TEN": ["Tennessee Titans", "Tennessee"],
    "WAS": ["Washington Commanders", "Washington"],
}

# flatten into reverse lookup once
NAME_TO_ABBR = {name: abbr for abbr, names in TEAM_ABBR.items() for name in names}

# standardize team names in scraped dfs
def_epa_df["team"] = def_epa_df["team"].map(NAME_TO_ABBR)
merged_third_down_df["team"] = merged_third_down_df["Team"].map(NAME_TO_ABBR)
merged_third_down_df.drop(columns=["Team"], inplace=True)
def_epa_df = def_epa_df.rename(columns={"EPA/Play": "def_epa_per_play"})
merged_third_down_df = merged_third_down_df.rename(columns={
    "2025_Offense": "off_third_down_pct",
    "2025_Defense": "def_third_down_pct"
})

In [77]:
# Merge all three DataFrames 
all_stats = cumulative_df.merge(def_epa_df, on='team', how="inner").merge(merged_third_down_df, on="team", how="inner")

In [78]:
print(all_stats.head(10))

  team  off_epa_per_play  points_for  points_against  turnover_diff  \
0  ARI          1.407971   20.666667       17.000000       0.666667   
1  ATL         -2.770102   14.000000       19.666667       0.333333   
2  BAL          6.213836   37.000000       32.000000       0.000000   
3  BUF          7.803691   34.000000       23.666667       1.000000   
4  CAR         -2.611105   20.666667       17.666667       0.000000   
5  CHI          0.191999   25.333333       31.000000       1.000000   
6  CIN         -5.970140   19.333333       30.333333      -1.333333   
7  CLE         -7.418851   15.333333       22.666667      -1.333333   
8  DAL          2.157078   24.666667       30.666667      -1.666667   
9  DEN         -0.643781   22.666667       21.333333       0.000000   

   def_epa_per_play off_third_down_pct def_third_down_pct  
0             -0.01             38.46%             34.62%  
1             -0.05             41.82%             34.09%  
2              0.14             40.91%